## FOR COMP90024 ASSIGNMENT 2
### BY TEAM 45:
    - William Chen 1400081
    - Petr Andreev 1375858
    - Rafsan Al Mamun 1407776
    - Ojaswi Dheer 1447227

# Lung Disease Analysis in Victoria
---------------------------------
This notebook explores lung disease data collected from SUDO across all of Australia. The dataset includes mortality rates, income levels, employment status, gender etc for asthma and COPD. The objective of this analysis is to gain insights into the relationship of demographics with lung diseases.

## Import Necessary Libraries:
----------------------------

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

## Load the Dataset:
--------------------

There are several datasets for this analysis that contains statistics for lung diseases across various factors.

Before proceeding with the analysis, we will load all the datasets.

In [2]:
# Set display options for pandas DataFrame
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', 1000)        # Set display width

In [4]:
# Load the datasets
male_lung_cancer_data = pd.read_csv('../data/Lung Disease Dataset/aihw_cimar_mortality_males_gccsa_2009.csv')
female_lung_cancer_data = pd.read_csv('../data/Lung Disease Dataset/aihw_cimar_mortality_females_gccsa_2009.csv')
all_lung_cancer_data = pd.read_csv('../data/Lung Disease Dataset/aihw_cimar_mortality_persons_gccsa_2009.csv')
census_by_cob_data = pd.read_csv('../data/Lung Disease Dataset/abs_2021census_g21a_aust_gccsa.csv')
census_by_inc_emp_data = pd.read_csv('../data/Lung Disease Dataset/abs_2021census_g21b_aust_gccsa.csv')